This is a scheduelr to incorporate new matches into the an excel file. 
The excel file would be reviewed and then added to db

In [40]:
import pandas as pd
from datetime import datetime
import config
import soccerdata as sd
import sql_connector
import schedule_helper as SH

In [2]:
FOTMOB_URL = config.getFotMobUrls()
FOTMOB_LEAGUE = config.getFotMobLeagueDict()
CONFIG = config.getGeneralConfig()

In [3]:
FOTMOB_LEAGUE

{'ENG-Premier League': 47,
 'ESP-La Liga': 87,
 'ITA-Serie A': 55,
 'GER-Bundesliga': 54,
 'FRA-Ligue 1': 53,
 'INT-World Cup': 77,
 'UCL': 42,
 'UEL': 73,
 'UECL': 10216,
 'Eredivisie': 57,
 'Pro_League': 40,
 'Primeira_Liga': 61}

In [4]:
leagues = sorted(list(FOTMOB_LEAGUE))

In [5]:
def getCurrentSeasonName(dt):
    month = dt.month
    yr = dt.year
    if month <=7 :
        return f"{(yr%2000)-1}-{yr%2000}",(yr%2000)-1
    else:
        return f"{yr%2000}-{(yr%2000)+1}",(yr%2000)

In [6]:
league = leagues[0]
season,df_season = getCurrentSeasonName(datetime.now())
season

'23-24'

In [7]:
last_date_info = []
for x in leagues :
    print("Working for league ",x)
    query = f"""
    SELECT competition,MAX(date) as last_match_date from match_schedule
    where competition = '{x}' AND season = '{season}'
    """
    # if x == 'Eredivisie':
    #     print(query)
    #     print()
    response = sql_connector.execute_select_query(query)[0]['last_match_date']
    if response:
        last_date_info.append({ x : response})


Working for league  ENG-Premier League
Working for league  ESP-La Liga
Working for league  Eredivisie
Working for league  FRA-Ligue 1
Working for league  GER-Bundesliga
Working for league  INT-World Cup
Working for league  ITA-Serie A
Working for league  Primeira_Liga
Working for league  Pro_League
Working for league  UCL
Working for league  UECL
Working for league  UEL


In [8]:
last_date_info

[{'ENG-Premier League': datetime.date(2024, 3, 17)},
 {'ESP-La Liga': datetime.date(2024, 3, 17)},
 {'Eredivisie': datetime.date(2024, 3, 17)},
 {'FRA-Ligue 1': datetime.date(2024, 3, 17)},
 {'GER-Bundesliga': datetime.date(2024, 3, 17)},
 {'ITA-Serie A': datetime.date(2024, 3, 17)},
 {'Primeira_Liga': datetime.date(2024, 3, 17)},
 {'Pro_League': datetime.date(2024, 3, 17)},
 {'UCL': datetime.date(2024, 3, 13)},
 {'UEL': datetime.date(2024, 3, 14)}]

In [ ]:
# item = last_date_info[-3:-2]
# item

In [9]:
def fetch_and_merge_current_season_dfs(league,current_season):
    WS_df,FB_df,FM_df = SH.fetch_all_dfs_for_season_of_league(league, latest_season=current_season)
    WS,FB,FM = SH.make_dfs_uniform(league,WS_df,FB_df,FM_df)
    team_alias = SH.get_team_alias()
    return SH.merge_and_save_df(league,WS,FB,FM,team_alias)
# df = fetch_and_merge_current_season_dfs(p[0],df_season)

In [27]:
def fetch_games_since_last(df,last_date):
    DF = df
    DF['date'] = pd.to_datetime(DF['date']).dt.date
    DF = DF[DF['date'] >= last_date]
    col_str = "("+",".join(list(DF.columns))+")"
    queries = []
    for i,(ind,x) in enumerate(DF.iterrows()):
        # print(i,ind)
        row = DF.iloc[i]
        whoscored_url = row['whoscored_url']
        fbref_url = row['fbref_url']
        fotmob_url = row['fotmob_url']
        
        all_urls_present = bool(whoscored_url) and bool(fbref_url) and bool(fotmob_url)
        
        if all_urls_present :
            value_str = "('"+"','".join(map(str, x))+"')"
            q = f'INSERT INTO match_schedule {col_str} VALUES {value_str}'.replace('.0','')
            queries.append((whoscored_url,fbref_url,fotmob_url,q))
        else:
            print("All IDs are not present : ",row['home_team'],row['away_team'],row['date'])
            print("WS: ",bool(whoscored_url),"FB: ",bool(fbref_url),"FM: ",bool(fotmob_url))
            print()
    return queries

In [28]:
insert_queries = []
for item in last_date_info:
    # print(item)
    # continue
    league_time_info = tuple(item.items())[0]
    df = fetch_and_merge_current_season_dfs(league_time_info[0],df_season)
    insert_queries += fetch_games_since_last(df,league_time_info[1])

[03/26/24 07:57:50] INFO     Saving cached data to C:\Users\Hp\soccerdata\data\WhoScored              _common.py:91

[03/26/24 07:57:51] INFO     Saving cached data to C:\Users\Hp\soccerdata\data\FBref                  _common.py:91

[03/26/24 07:58:16] INFO     Scraping game schedule from                                           whoscored.py:428
                             https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/9618/Engl                 
                             and-Premier-League                                                                    

                    INFO     Scraping game schedule for Jumamosi, Mac 2 2024                       whoscored.py:327

[03/26/24 07:58:17] INFO     Scraping game schedule for Jumapili, Mac 3 2024                       whoscored.py:327

...

[03/26/24 08:11:31] INFO     Scraping game schedule for Alhamisi, Ago 10 2023                      whoscored.py:327

                    INFO     Scraping game schedule for Alhamisi, Ago 17 2023                      whoscored.py:327

                    INFO     Scraping game schedule for Alhamisi, Ago 24 2023                      whoscored.py:327

[03/26/24 08:11:32] INFO     Scraping game schedule for Alhamisi, Ago 31 2023                      whoscored.py:327

Returning 1 DFs combination
Preparing Fetched DataFrames ... 
UEL
Season 23-24 | Group Stage Start 2023-09-21 16:45:00+00:00
WS : (170, 9) | FB : (136, 19) | FM : (141, 8)
Merging Fetched DataFrames ... 
(136, 9)


In [ ]:
insert_queries = fetch_games_since_last(df,league_time_info[1])

In [29]:
insert_queries

[('https://1xbet.whoscored.com/Matches/1729432/Live/England-Premier-League-2023-2024-West-Ham-Aston-Villa',
  '/en/matches/fc356c59/West-Ham-United-Aston-Villa-March-17-2024-Premier-League',
  '/matches/west-ham-united-vs-aston-villa/2yexzh#4193819',
  "INSERT INTO match_schedule (competition,season,stage,date,home_team,away_team,score,whoscored_id,whoscored_url,fbref_id,fbref_url,fotmob_id,fotmob_url) VALUES ('ENG-Premier League','23-24','None','2024-03-17','West Ham','Aston Villa','1–1','1729432','https://1xbet.whoscored.com/Matches/1729432/Live/England-Premier-League-2023-2024-West-Ham-Aston-Villa','fc356c59','/en/matches/fc356c59/West-Ham-United-Aston-Villa-March-17-2024-Premier-League','4193819','/matches/west-ham-united-vs-aston-villa/2yexzh#4193819')"),
 ('https://1xbet.whoscored.com/Matches/1734989/Live/Spain-LaLiga-2023-2024-Sevilla-Celta-Vigo',
  '/en/matches/677dc25e/Sevilla-Celta-Vigo-March-17-2024-La-Liga',
  '/matches/sevilla-vs-celta-vigo/2qqwgg#4205629',
  "INSERT INTO 

In [39]:
err_ind = []
dfs = []
qqq = []
for i,(WS_url,FB_url,FM_url,q) in enumerate(insert_queries):
    try:
        fetch_query = f"select * from match_schedule where whoscored_url like'%{WS_url}%' or"
        fetch_query += f" fbref_url like'%{FB_url}%' or fotmob_url like '%{FM_url}%'; "
        fetch_query = fetch_query.replace(".0",'')
        l = sql_connector.execute_select_query(fetch_query)
        print("Done Query No : ",i+1,len(l))
        
        if len(l) !=0:
            if len(l) > 1 :
                dfs.append(pd.DataFrame(l))
            continue
        else:
            qqq.append(q)
        # sql_connector.execute_insert_query(q)
        # print(l,type(l))
        print()
    except Exception as e:
        err_ind.append((i,str(e)))

Done Query No :  1 1
Done Query No :  2 1
Done Query No :  3 1
Done Query No :  4 1
Done Query No :  5 1
Done Query No :  6 1
Done Query No :  7 1
Done Query No :  8 1
Done Query No :  9 1
Done Query No :  10 1
Done Query No :  11 1
Done Query No :  12 1
Done Query No :  13 1
Done Query No :  14 1
Done Query No :  15 1
Done Query No :  16 1
Done Query No :  17 1
Done Query No :  18 1
Done Query No :  19 1
Done Query No :  20 1
Done Query No :  21 1
Done Query No :  22 1
Done Query No :  23 1
Done Query No :  24 2
Done Query No :  25 1
Done Query No :  26 1
Done Query No :  27 1
Done Query No :  28 1
Done Query No :  29 1
Done Query No :  30 1
Done Query No :  31 1
Done Query No :  32 1
Done Query No :  33 1
Done Query No :  34 1
Done Query No :  35 1
Done Query No :  36 1
Done Query No :  37 1
Done Query No :  38 1
Done Query No :  39 2
Done Query No :  40 2
Done Query No :  41 2
Done Query No :  42 1
Done Query No :  43 1


In [31]:
[(i,x.shape) for i,x in enumerate(dfs)]

[(0, (2, 15)),
 (1, (2, 15)),
 (2, (2, 15)),
 (3, (2, 15)),
 (4, (2, 15)),
 (5, (2, 15)),
 (6, (2, 15))]

In [32]:
abc = pd.concat(dfs)

In [33]:
abc['whoscored_url'].value_counts(),\
abc['fbref_url'].value_counts(),\
abc['fotmob_url'].value_counts(),

(whoscored_url
 https://1xbet.whoscored.com/Matches/1748587/Live/Portugal-Liga-Portugal-2023-2024-Chaves-Vitoria-de-Guimaraes    2
 https://1xbet.whoscored.com/Matches/1748575/Live/Portugal-Liga-Portugal-2023-2024-Moreirense-Arouca              2
 https://1xbet.whoscored.com/Matches/1748598/Live/Portugal-Liga-Portugal-2023-2024-Casa-Pia-AC-Benfica            2
 https://1xbet.whoscored.com/Matches/1748578/Live/Portugal-Liga-Portugal-2023-2024-Sporting-CP-Boavista           2
 https://1xbet.whoscored.com/Matches/1775714/Live/Europe-Europa-League-2023-2024-West-Ham-Freiburg                1
 https://1xbet.whoscored.com/Matches/1805544/Live/Europe-Europa-League-2023-2024-West-Ham-Freiburg                1
 https://1xbet.whoscored.com/Matches/1775770/Live/Europe-Europa-League-2023-2024-Atalanta-Sporting-CP             1
 https://1xbet.whoscored.com/Matches/1805546/Live/Europe-Europa-League-2023-2024-Atalanta-Sporting-CP             1
 https://1xbet.whoscored.com/Matches/1775793/Live/Europe-

In [36]:
qqq

[]